In [ ]:
import os
import time

import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds
from google.colab import drive
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
tfds.disable_progress_bar()

train_ds, validation_ds, test_ds = tfds.load(
    "cats_vs_dogs",
    # Reserve 10% for validation and 10% for test
    split=["train[:40%]", "train[40%:50%]", "train[50%:60%]"],
    as_supervised=True,  # Include labels
)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incomplete0TYX6V/cats_vs_dogs-train.tfrecord
Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [ ]:
drive.mount('/content/gdrive')
path = "/content/gdrive/My Drive/colab/training_2/tfjs_cat_dog" 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
size = (224, 224)
BFS = 10
BTS = 32

train2_ds = train_ds.map(lambda x, y: (tf.image.resize(x, size), y ))
val2_ds = validation_ds.map(lambda x, y: (tf.image.resize(x, size), y ))
test2_ds = test_ds.map(lambda x, y: (tf.image.resize(x, size), y ))

train2_ds = train2_ds.cache().batch(BTS).prefetch(buffer_size=BFS)
val2_ds = val2_ds.cache().batch(BTS).prefetch(buffer_size=BFS)
test2_ds = test2_ds.cache().batch(BTS).prefetch(buffer_size=BFS)

In [ ]:
checkpoint_path = "/content/gdrive/My Drive/colab/training_2/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

## Use a Sequential Model to be able to use with TensorFlowJS

For a comparison of different CNN-architecture see [this explanation](https://www.kaggle.com/shivamb/cnn-architectures-vgg-resnet-inception-tl#1.4-Resnets).
Due to its simple linear series of layers, VGG is the only net I was able to use with TFJS.
Xception, MobileNet, ResNet and Inception all contain some parallel processing parts which is why they cannot be disassembled and recomposed as needed for the creation of a Sequential model which is readable in TFJS.

From the [documentation](https://keras.io/guides/sequential_model/#transfer-learning-with-a-sequential-model):
>A Sequential model is not appropriate when:

>    - Your model has multiple inputs or multiple outputs
>   - Any of your layers has multiple inputs or multiple outputs
>    - You need to do layer sharing
>    - You want non-linear topology (e.g. a residual connection, a multi-branch model)


In [ ]:

b_model = keras.applications.VGG19(
    weights="imagenet",
    input_shape=(224, 224, 3),
    include_top=False,
) 

# Freeze the base_model
b_model.trainable = False
layers = b_model.layers
model2 = keras.Sequential()
model2.add(keras.Input(shape=(224, 224, 3)))
model2.add(keras.layers.BatchNormalization())

# remove first (input) layer
# it's a functional layer which causes tensoflowjs to crash 
print(len(layers), len(layers[1:]))
for layer in layers[1:]:
  layer.trainable = False
  model2.add(layer)

model2.add(keras.layers.GlobalAveragePooling2D())
model2.add(keras.layers.Dropout(0.2))
model2.add(keras.layers.Dense(1, activation="selu"))
model2.load_weights(checkpoint_path)
model2.build()
model2.summary()

22 21
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 224, 224, 3)       12        
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)

In [ ]:
start_1 = time.time()

model2.compile(
    optimizer=keras.optimizers.SGD(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 20 # sigmoid: 0.9441
model2.fit(train2_ds, epochs=epochs, validation_data=val2_ds, callbacks=[cp_callback])

print((time.time() - start_1) // 60)

Epoch 1/20
    291/Unknown - 98s 336ms/step - loss: 0.4914 - binary_accuracy: 0.7556
Epoch 00001: saving model to /content/gdrive/My Drive/colab/training_2/cp.ckpt
291/291 [==============================] - 113s 388ms/step - loss: 0.4914 - binary_accuracy: 0.7556 - val_loss: 0.3627 - val_binary_accuracy: 0.9067
Epoch 2/20
291/291 [==============================] - ETA: 0s - loss: 0.3379 - binary_accuracy: 0.8927
Epoch 00002: saving model to /content/gdrive/My Drive/colab/training_2/cp.ckpt
291/291 [==============================] - 111s 383ms/step - loss: 0.3379 - binary_accuracy: 0.8927 - val_loss: 0.2923 - val_binary_accuracy: 0.9381
Epoch 3/20
291/291 [==============================] - ETA: 0s - loss: 0.2872 - binary_accuracy: 0.9197
Epoch 00003: saving model to /content/gdrive/My Drive/colab/training_2/cp.ckpt
291/291 [==============================] - 112s 384ms/step - loss: 0.2872 - binary_accuracy: 0.9197 - val_loss: 0.2557 - val_binary_accuracy: 0.9407
Epoch 4/20
291/291 [=====

In [ ]:
start_2 = time.time()

for lyr in model2.layers[2:23]:
  lyr.trainable = True

# It's important to recompile the model after making any changes to the
# `trainable` attribute of any inner layer, so that changes are taken into account
model2.compile(
    optimizer=keras.optimizers.SGD(1e-5),  # Low learning rate
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 10
model2.fit(train2_ds, epochs=epochs, validation_data=val2_ds, callbacks=[cp_callback])
print((time.time() - start_2) // 60)

Epoch 1/10
      2/Unknown - 1s 462ms/step - loss: 0.1374 - binary_accuracy: 0.9688WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.2810s vs `on_train_batch_end` time: 0.6323s). Check your callbacks.


    291/Unknown - 274s 940ms/step - loss: 0.1507 - binary_accuracy: 0.9686
Epoch 00001: saving model to /content/gdrive/My Drive/colab/training_2/cp.ckpt
291/291 [==============================] - 297s 1s/step - loss: 0.1507 - binary_accuracy: 0.9686 - val_loss: 0.1395 - val_binary_accuracy: 0.9776
Epoch 2/10
291/291 [==============================] - ETA: 0s - loss: 0.1475 - binary_accuracy: 0.9690
Epoch 00002: saving model to /content/gdrive/My Drive/colab/training_2/cp.ckpt
291/291 [==============================] - 286s 982ms/step - loss: 0.1475 - binary_accuracy: 0.9690 - val_loss: 0.1387 - val_binary_accuracy: 0.9776
Epoch 3/10
291/291 [==============================] - ETA: 0s - loss: 0.1469 - binary_accuracy: 0.9701
Epoch 00003: saving model to /content/gdrive/My Drive/colab/training_2/cp.ckpt
291/291 [==============================] - 285s 979ms/step - loss: 0.1469 - binary_accuracy: 0.9701 - val_loss: 0.1379 - val_binary_accuracy: 0.9776
Epoch 4/10
291/291 [==================

In [ ]:
image_batch, label_batch = test2_ds.as_numpy_iterator().next()
predictions = model2.predict(image_batch).flatten()
# predictions = tf.where(predictions < 0.5, 0, 1)
print(f'{label_batch}: Labels')
print(f'{predictions}: Predictions')

[0 1 1 1 1 1 1 1 0 1 0 1 0 0 0 1 0 1 0 1 1 1 0 0 0 0 0 0 0 0 1 0]: Labels
[-1.7512559   7.5966434  14.456075    8.774034    1.9429191   5.5053253
  0.98495924  3.9802139  -1.7439785   6.799868   -1.6845     16.236485
 -1.7520068  -1.7572677  -1.7299792   1.4699501  -1.7503388  14.003305
 -1.7563033   4.27554     9.00075     8.491771   -1.7508816  -1.7473652
 -1.7457969  -1.7578459  -1.7575895  -1.5759106  -1.3535689  -1.730011
  8.585855   -1.7567315 ]: Predictions


In [ ]:
funcs = {
    "selu": tf.nn.selu,
    "relu": tf.nn.relu,
    "relu6": tf.nn.relu6,
    "elu": tf.nn.elu,
    #"swish": tf.nn.swish,
    "leaky-relu": tf.nn.leaky_relu,
    #"sigmoid": tf.nn.sigmoid,
    #"softsign": tf.nn.softsign,
    # not applicable: 
    #"crelu": tf.nn.crelu,
    #"softmax": tf.nn.softmax,
    #"log_softmax": tf.nn.log_softmax,
}

image_batch, label_batch = test2_ds.as_numpy_iterator().next()
predictions = model2.predict(image_batch).flatten()
predictions = tf.convert_to_tensor(predictions)
print(f'{label_batch}: Labels')

for func in funcs.keys():
    function = funcs[func]
    predictions1 = function(predictions)
    predictions1 = tf.where(predictions1 < 0.5, 0, 1)
    print(f'{predictions1.numpy()}: Predictions {func}')

[0 1 1 1 1 1 1 1 0 1 0 1 0 0 0 1 0 1 0 1 1 1 0 0 0 0 0 0 0 0 1 0]: Labels
[0 1 1 1 1 1 1 1 0 1 0 1 0 0 0 1 0 1 0 1 1 1 0 0 0 0 0 0 0 0 1 0]: Predictions selu
[0 1 1 1 1 1 1 1 0 1 0 1 0 0 0 1 0 1 0 1 1 1 0 0 0 0 0 0 0 0 1 0]: Predictions relu
[0 1 1 1 1 1 1 1 0 1 0 1 0 0 0 1 0 1 0 1 1 1 0 0 0 0 0 0 0 0 1 0]: Predictions relu6
[0 1 1 1 1 1 1 1 0 1 0 1 0 0 0 1 0 1 0 1 1 1 0 0 0 0 0 0 0 0 1 0]: Predictions elu
[0 1 1 1 1 1 1 1 0 1 0 1 0 0 0 1 0 1 0 1 1 1 0 0 0 0 0 0 0 0 1 0]: Predictions leaky-relu


In [ ]:
class_names=['cat','dog']
img = keras.preprocessing.image.load_img(
    "/Users/natalie/Downloads/iu-4.jpeg", target_size=(150,150)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array, steps=1)
predictions = tf.nn.sigmoid(predictions)
predictions1 = tf.where(predictions < 0.5, 0, 1)
print(class_names[predictions1.numpy()[0][0]])

## Saving model (for TFJS)

In [ ]:
!pip install tensorflowjs --quiet

     |████████████████████████████████| 71kB 6.3MB/s 
     |████████████████████████████████| 137.3MB 92kB/s 
     |████████████████████████████████| 92kB 12.7MB/s 
     |████████████████████████████████| 256kB 54.0MB/s 
     |████████████████████████████████| 921kB 53.6MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


### should work, but doesn't

In [ ]:
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(model2, "/content/gdrive/My Drive/colab/training_2/tfjs_model")

/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:123: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)


### Finally the right way

follow the solution of this guy: https://github.com/tensorflow/tfjs/issues/3772#issuecomment-672195520

In [ ]:
model2.save('/content/gdrive/My Drive/colab/test.h5')